In [1]:
include("lib/all.jl");

In [2]:
particles = make_particles(2^20);
t = Tree(particles);

In [3]:
@time group!(t, 2);

In [4]:
@time inform!(t)

  0.126665 seconds (13.82 k allocations: 593.706 KB, 45.41% gc time)


In [5]:
test_in_cell_mass(t)

In [6]:
ax = zeros(length(particles))
ay = zeros(length(particles))
az = zeros(length(particles));

In [7]:
@time get_all_accel!(t, 0.5, 1.0e-157, ax,ay,az)

 12

In [9]:
@time get_all_accel_rel!(t, 0.005, 1.0e-157, ax,ay,az)

 14

In [8]:
perf(particles, ax, ay, az, 5000, 1.0e-157)

(pex,pey,pez) = (41.12736167915755,57.8456982951611,38.18144572312316)


In [10]:
perf(particles, ax, ay, az, 5000, 1.0e-157)

(pex,pey,pez) = (59.430477674794744,52.79237577542211,65.36517586559474)


(59.430477674794744,52.79237577542211,65.36517586559474)